# Build a Retrieval Augmented Generation (RAG) App

##### **NOTE:** *The tutorial content in this notebook are from LangChain tutorials. However, the implementation of the RAG pipeline in this notebook is not from LangChain, but follows a similar approach.*

One of the most powerful applications enabled by LLMs is sophisticated question-answering (Q&A) chatbots. These are applications that can answer questions about specific source information. These applications use a technique known as Retrieval Augmented Generation, or [RAG](/docs/concepts/rag/).

LangChain offers a multi-part tutorial:

- [Part 1](https://python.langchain.com/docs/tutorials/rag/) (similar to this guide) introduces RAG and walks through a minimal implementation.
- [Part 2](https://python.langchain.com/docs/tutorials/qa_chat_history) extends the implementation to accommodate conversation-style interactions and multi-step retrieval processes.

This tutorial will show how to build a simple Q&A application
over a text data source. If you're already familiar with basic retrieval, you might also be interested in
this [high-level overview of different retrieval techniques](https://python.langchain.com/docs/concepts/retrieval/).

**Note**: Here we focus on Q&A for unstructured data. If you are interested for RAG over structured data, check out our tutorial on doing [question/answering over SQL data](https://python.langchain.com/docs/tutorials/sql_qa/).

## Overview
A typical RAG application has two main components:

**Indexing**: a pipeline for ingesting data from a source and indexing it. *This usually happens offline.*

**Retrieval and generation**: the actual RAG chain, which takes the user query at run time and retrieves the relevant data from the index, then passes that to the model.

Note: the indexing portion of this tutorial will largely follow the [semantic search tutorial](https://python.langchain.com/docs/tutorials/retrievers).

The most common full sequence from raw data to answer looks like:

### Indexing
1. **Load**: First we need to load our data. This is done with [Document Loaders](https://python.langchain.com/docs/concepts/document_loaders).
2. **Split**: [Text splitters](https://python.langchain.com/docs/concepts/text_splitters) break large `Documents` into smaller chunks. This is useful both for indexing data and passing it into a model, as large chunks are harder to search over and won't fit in a model's finite context window.
3. **Store**: We need somewhere to store and index our splits, so that they can be searched over later. This is often done using a [VectorStore](https://python.langchain.com/docs/concepts/vectorstores) and [Embeddings](https://python.langchain.com/docs/concepts/embedding_models) model.

![index_diagram](https://github.com/langchain-ai/langchain/blob/master/docs/static/img/rag_indexing.png?raw=1)

### Retrieval and generation
4. **Retrieve**: Given a user input, relevant splits are retrieved from storage using a [Retriever](https://python.langchain.com/docs/concepts/retrievers).
5. **Generate**: A [ChatModel](https://python.langchain.com/docs/concepts/chat_models) / [LLM](/docs/concepts/text_llms) produces an answer using a prompt that includes both the question with the retrieved data

![retrieval_diagram](https://github.com/langchain-ai/langchain/blob/master/docs/static/img/rag_retrieval_generation.png?raw=1)

Once we've indexed our data, we will use [LangGraph](https://langchain-ai.github.io/langgraph/) as our orchestration framework to implement the retrieval and generation steps.

## Setup

### Jupyter Notebook

This and other tutorials are perhaps most conveniently run in a [Jupyter notebooks](https://jupyter.org/). Going through guides in an interactive environment is a great way to better understand them. See [here](https://jupyter.org/install) for instructions on how to install.

### Installation

This tutorial requires these langchain dependencies:

import Tabs from '@theme/Tabs';
import TabItem from '@theme/TabItem';
import CodeBlock from "@theme/CodeBlock";

<Tabs>
  <TabItem value="pip" label="Pip" default>
  

In [ ]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph
%pip install --quiet --upgrade langchain-openai
%pip install --quiet langchain_chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.2/153.2 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.5/443.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... d

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


## Detailed walkthrough

Let’s go through our code step-by-step to really understand what’s going on.

## 1. Indexing {#indexing}

:::note

This section is an abbreviated version of the content in the [semantic search tutorial](/docs/tutorials/retrievers).
If you're comfortable with [document loaders](/docs/concepts/document_loaders), [embeddings](/docs/concepts/embedding_models), and [vector stores](/docs/concepts/vectorstores),
feel free to skip to the next section on [retrieval and generation](/docs/tutorials/rag/#orchestration).

:::

### 2. Loading documents

We need to first load the blog post contents. We can use
[DocumentLoaders](/docs/concepts/document_loaders)
for this, which are objects that load in data from a source and return a
list of
[Document](https://python.langchain.com/api_reference/core/documents/langchain_core.documents.base.Document.html)
objects.

In this case we’ll use the
[WebBaseLoader](/docs/integrations/document_loaders/web_base),
which uses `urllib` to load HTML from web URLs and `BeautifulSoup` to
parse it to text. We can customize the HTML -\> text parsing by passing
in parameters into the `BeautifulSoup` parser via `bs_kwargs` (see
[BeautifulSoup
docs](https://beautiful-soup-4.readthedocs.io/en/latest/#beautifulsoup)).
In this case only HTML tags with class “post-content”, “post-title”, or
“post-header” are relevant, so we’ll remove all others.

### 3. Retrieve relevant documents
Now that we have the vector database, we can use it to augment a query, serving as context information. The retriever will output the top-k relevant documents that are related to the query.

### 4. Combine retrieved docs with query
The retrieved document and query are both passed as input to the LLM. Basically, the LLM uses the documents to understand the context of the user query and rendrers answers. It's a good practice to compare results without using the RAG approach.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

In [ ]:
import os
persistent_directory = os.path.join("db", "chroma_db")

# Check if the Chroma vector store already exists
if not os.path.exists(persistent_directory):
    print("Persistent directory does not exist. Initializing vector store...")

    # Read the text content from the file
    # Load and chunk contents of the blog
    loader = WebBaseLoader(
        web_paths=("https://medium.com/quantonation/a-beginners-guide-to-high-performance-computing-ae70246a7af",
                   "https://www.ibm.com/think/topics/hpc",
                   "https://algocademy.com/blog/high-performance-computing-hpc-optimizing-computational-tasks-for-speed-and-efficiency/"),
        bs_kwargs=dict(
            parse_only=bs4.SoupStrainer(
                class_=("post-content", "post-title", "post-header")
            )
        ),
    )
    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    docs = text_splitter.split_documents(documents)

    # Display information about the split documents
    print("\n--- Document Chunks Information ---")
    print(f"Number of document chunks: {len(docs)}")

    # Create embeddings
    print("\n--- Creating embeddings ---")
    embeddings = OpenAIEmbeddings(
        model="text-embedding-3-small"
    )  # Update to a valid embedding model if needed
    print("\n--- Finished creating embeddings ---")

    # Create the vector store and persist it automatically
    print("\n--- Creating vector store ---")
    db = Chroma.from_documents(
        docs, embeddings, persist_directory=persistent_directory)
    print("\n--- Finished creating vector store ---")

else:
    print("Vector store already exists. No need to initialize.")

Persistent directory does not exist. Initializing vector store...

--- Document Chunks Information ---
Number of document chunks: 63

--- Creating embeddings ---

--- Finished creating embeddings ---

--- Creating vector store ---

--- Finished creating vector store ---


In [ ]:
print(f"Sample chunk:\n{docs[0].page_content}\n")

Sample chunk:
LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:

Planning

Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.
Reflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.


Memory



In [ ]:
print(f"Total characters: {len(docs[0].page_content)}")

Total characters: 969


In [ ]:
# Load the existing vector store with the embedding function
db = Chroma(persist_directory=persistent_directory,
            embedding_function=embeddings)

query = input("Your question: ")

# Retrieve relevant documents based on the query
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 3, "score_threshold": 0.3},
)
relevant_docs = retriever.invoke(query)

# Display the relevant results with metadata
print("\n--- Relevant Documents ---")
for i, doc in enumerate(relevant_docs, 1):
    print(f"Document {i}:\n{doc.page_content}\n")
    if doc.metadata:
        print(f"Source: {doc.metadata.get('source', 'Unknown')}\n")

Your question: What are agengts?

--- Relevant Documents ---
Document 1:
They also discussed the risks, especially with illicit drugs and bioweapons. They developed a test set containing a list of known chemical weapon agents and asked the agent to synthesize them. 4 out of 11 requests (36%) were accepted to obtain a synthesis solution and the agent attempted to consult documentation to execute the procedure. 7 out of 11 were rejected and among these 7 rejected cases, 5 happened after a Web search while 2 were rejected based on prompt only.
Generative Agents Simulation#
Generative Agents (Park, et al. 2023) is super fun experiment where 25 virtual characters, each controlled by a LLM-powered agent, are living and interacting in a sandbox environment, inspired by The Sims. Generative agents create believable simulacra of human behavior for interactive applications.
The design of generative agents combines LLM with memory, planning and reflection mechanisms to enable agents to behave con

In [ ]:
# Combine the query and the relevant document contents
combined_input = (
    "Here are some documents that might help answer the question: "
    + query
    + "\n\nRelevant Documents:\n"
    + "\n\n".join([doc.page_content for doc in relevant_docs])
    + "\n\nPlease provide a rough answer based only on the provided documents. If the answer is not found in the documents, respond with 'I'm not sure'."
)

# Create a ChatOpenAI model
llm = ChatOpenAI(model="gpt-4.1-mini")

# Define the messages for the model
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content=combined_input),
]

# Invoke the model with the combined input
result = llm.invoke(messages)

# Display the full result and content only
print("\n--- Generated Response ---")
# print("Full result:")
# print(result)
# print(result.content)

# Display the content as markdown for better formatting
from IPython.display import display, Markdown

display(Markdown(result.content))


--- Generated Response ---


Based on the provided documents, agents, particularly generative agents, are LLM-powered autonomous entities that simulate human-like behavior in interactive environments. They combine large language models (LLMs) with mechanisms like memory, planning, and reflection to behave based on past experiences and interact with other agents. These agents can perform tasks independently, such as synthesizing chemical weapon agents or engaging in social behaviors like conversations and event coordination in simulations. They use natural language interfaces to communicate and operate, although reliability issues with formatting and instruction adherence exist.

## Conversational chatBot

In [ ]:
# Create a ChatOpenAI model
model = ChatOpenAI(model="gpt-4.1-mini")

chat_history = []  # Use a list to store messages

# Set an initial system message (optional)
system_message = SystemMessage(content="You are a helpful assistant.")
chat_history.append(system_message)  # Add system message to chat history

# Chat loop
while True:
    query = input("You: ")
    if query.lower() == "exit":
        break
    chat_history.append(HumanMessage(content=query))  # Add user message

    # Get AI response using history
    result = model.invoke(chat_history)
    response = result.content
    chat_history.append(AIMessage(content=response))  # Add AI message

    # print(f"ChatBot: {response}")

    print('\n --- AI Generated Response ---\n')

    display(Markdown(response))

    print('\n')


# print("---- Message History ----")
# print(chat_history)

You: Who are you?

 --- AI Generated Response ---



Hello! I am ChatGPT, an AI language model created by OpenAI. I'm here to help answer your questions, provide information, and assist with a variety of tasks. How can I assist you today?



You: Tell me about ALCF

 --- AI Generated Response ---



Certainly! The ALCF, or Argonne Leadership Computing Facility, is a prominent high-performance computing (HPC) center located at Argonne National Laboratory in Lemont, Illinois. It provides researchers with some of the most powerful supercomputing resources available to support scientific research across various disciplines.

**Key Highlights about ALCF:**

- **Purpose:** The ALCF is dedicated to enabling breakthrough scientific discoveries by providing access to extremely high-performance computing resources.
- **Supercomputers:** It operates some of the world's most advanced supercomputers, such as the "Theta" and "Aurora" systems, designed for large-scale computational research.
- **Research Areas:** The facility supports research in quantum physics, materials science, biology, climate modeling, energy research, and more.
- **User Community:** Researchers from universities, government agencies, and industry can apply for access through competitive proposals to utilize ALCF’s resources.
- **Collaboration and Support:** The ALCF offers support and collaboration opportunities to scientists to help optimize their codes and workflows for supercomputing environments.

If you'd like more specific information or recent updates about the ALCF, let me know!

KeyboardInterrupt: Interrupted by user